<a href="https://colab.research.google.com/github/YorkJong/news-digest/blob/main/notebooks/news_clip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

這支 colab notebook 是為了選取剪輯 [news-digest](https://news-digest.vercel.app/) 網站當天的新聞而寫的。只根據新聞分類(catetories)剪輯。

執行步驟如下：
1. 按下 Install 下那個 code cell 前面的 ► 按鈕來安裝用到的模組
2. 根據需要，填表執行後面的 Forms 或 Widgets UI

需要更細的檢索查詢，有如下兩個選擇
- 直接在 [news-digest 網站](https://news-digest.vercel.app/)上搜尋或點選 hash-tags
- 或者點擊 [news_query.ipynb](https://colab.research.google.com/github/YorkJong/news-digest/blob/main/notebooks/news_query.ipynb) 執行多條件檢索


### Install 

In [21]:
import os

fns = ['clip.py', 'line.py', 'op.py']

for fn in fns:
    if os.path.exists(fn):
        os.remove(fn)
for fn in fns:
    url = f'https://raw.githubusercontent.com/YorkJong/news-digest/main/api/{fn}'
    !wget $url

import clip
import line

#-------------------------------------------------------------------------------
import ipywidgets as widgets

def create_menu(content):
    options = clip.get_categories(content)
    categories = widgets.SelectMultiple(
        options=options,
        value=[options[0]],
        rows=len(options),
        description='Categories',
        disabled=False,
    )

    with_hashtags = widgets.Checkbox(
        value=True,
        description='with hashtags',
        disabled=False,
    )

    with_headers = widgets.Checkbox(
        value=True,
        description='with headers',
        disabled=False,
    )

    flags = widgets.VBox([with_hashtags, with_headers])
    output = widgets.Output()
    display(widgets.HBox([categories, flags]), output)

    def on_value_change(change):
        with output:
            #print(change['new'])
            output.clear_output()
            lines = clip.get_lines_of_categories(categories.value, content, with_hashtags.value, with_headers.value)
            print('\n'.join(lines))

    categories.observe(on_value_change, names='value')
    with_hashtags.observe(on_value_change, names='value')
    with_headers.observe(on_value_change, names='value')

--2023-03-26 03:36:16--  https://raw.githubusercontent.com/YorkJong/news-digest/main/api/clip.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6800 (6.6K) [text/plain]
Saving to: ‘clip.py’

clip.py             100%[===================>]   6.64K  --.-KB/s    in 0s      

2023-03-26 03:36:16 (76.8 MB/s) - ‘clip.py’ saved [6800/6800]

--2023-03-26 03:36:16--  https://raw.githubusercontent.com/YorkJong/news-digest/main/api/line.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3425 (3.3K) [text/plain]
Saving to: ‘line.py’

line.py 

### Select Categories with the Colab Forms

In [2]:
#@title Single Selection { run: "auto", display-mode: "form" }
category = "AI" #@param ["Tesla & SpaceX; Vehicle", "Tech Titans", "Finance", "Taiwan", "Crypto", "Technology", "AI"] {allow-input: true}
with_hashtags = False #@param {type:"boolean"}
with_headers = False #@param {type:"boolean"}

content = clip.get_latest_journal()
lines = clip.get_lines_of_categories([category], content, with_hashtags, with_headers)
text = '\n'.join(lines)
print(text)

- [GPT-4 能做什麼？微軟發表GPT-4的完整測評，並且認為這就是第一版的通用人工智慧](https://www.techbang.com/posts/104938-microsoft-gpt-4-ai)
- [OpenAI 開放 ChatGPT 與第三方 API 資源串接 提供更即時、精準的服務](https://www.cool3c.com/article/191088)
- [官方版 ChatGPT plugins 開放搜尋最新資訊！懂修圖、程式碼視覺化還支援上傳檔案！](https://www.kocpc.com.tw/archives/485595)
-


In [ ]:
#@title Multiple Selections { run: "auto", display-mode: "form" }
first_category = "Tesla & SpaceX; Vehicle" #@param ["Tesla & SpaceX; Vehicle", "Tech Titans", "Finance", "Taiwan", "Crypto", "Technology", "AI"] {allow-input: true}
last_category = "Finance" #@param ["Tesla & SpaceX; Vehicle", "Tech Titans", "Finance", "Taiwan", "Crypto", "Technology", "AI"] {allow-input: true}
with_hashtags = False #@param {type:"boolean"}
with_headers = True #@param {type:"boolean"}

content = clip.get_latest_journal()
categories = clip.get_categories(content)
categories = clip.get_sublist(categories, first_category, last_category)
lines = clip.get_lines_of_categories(categories, content, with_hashtags, with_headers)
text = '\n'.join(lines)
print(text)


### Tesla & SpaceX; Vehicle
- [Tesla汽車引發價格戰 恐令大陸汽車市場大洗牌](https://www.chinatimes.com/realtimenews/20230323005342-260409)
- [太狂了！Tesla 特斯拉車主拆了 Model 3 ，用零件打造「CyberRoadster」跑車](https://today.line.me/tw/v2/article/0MOeanp)
- [BTCC特斯拉通證化合約 交易量超越狗狗幣](https://news.cnyes.com/news/id/5125754)
- [馬斯克新金主？中東國家也想投資SpaceX - 自由財經](https://ec.ltn.com.tw/article/breakingnews/4248893)
- [豐田艱難路（4）繼續挑戰氫能車  日經中文網](https://zh.cn.nikkei.com/industry/icar/51809-2023-03-24-05-03-00.html)
-
### Tech Titans
- [NVIDIA CEO：延續摩爾定律優勢得靠AI人工智慧編寫軟體](https://www.4gamers.com.tw/news/detail/57402/nvidia-ceo-jensen-huang-talk-about-moore-s-law-after-gtc-2023-march)
- [NVIDIA CEO 黃仁勳 希望繼續領導3或4個10年](https://www.coolaler.com/index/nvidia-ceo-黃仁勳-希望繼續領導3或4個10年/)
- [NVIDIA、ASML、台積電與 Synopsys 攜手合力運算式微影技術挑戰物理極限](https://news.xfastest.com/nvidia/125744/nvidia-asml-tsmc-synopsys-culitho/)
- [微軟高層 ：Bard 跟 Bing 還落後很多！](https://www.newmobilelife.com/2023/03/24/mikhail-parakhin-to-comment-bard/)
- [醫療行政負荷大解放！微軟子公司推出整合 GPT-4 的門診內容自動轉寫技術

### Select categories with Jupyter Widgets
- 按下面那個 code cell 前面的 ► 按鈕，以列出完整的分類(Catetories)選單
- macOS 下操作可按Cmd後用滑鼠複選（Windows 系統則按下Ctrl鍵後以滑鼠複選）
  - 全選可按 Cmd+A (macOS) 或者 Ctrl+A (Windows)

In [22]:
create_menu(clip.get_latest_journal())

Output()

### See Also
- [news_query.ipynb (from GitHub)](https://colab.research.google.com/github/YorkJong/news-digest/blob/main/notebooks/news_query.ipynb)